In [ ]:
import random
import pickle
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import Sequential , load_model
from tensorflow.keras.layers import LSTM , Dense , Activation
from tensorflow.keras.optimizers import RMSprop

In [ ]:
text_df  = pd.read_csv('fake_or_real_news.csv')

In [ ]:
text_df.head()

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
text = list(text_df.text.values)
joined_text = " ".join(text)

with open("joined_text.txt" , "w" , encoding = "utf-8" , errors = "ignore") as f :
  f.write(joined_text)

In [ ]:
partial_text = joined_text[:70000]

In [ ]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [ ]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index , token in enumerate(unique_tokens)}

In [ ]:
unique_token_index

{np.str_('0'): 0,
 np.str_('000'): 1,
 np.str_('1'): 2,
 np.str_('10'): 3,
 np.str_('100'): 4,
 np.str_('106'): 5,
 np.str_('10th'): 6,
 np.str_('11'): 7,
 np.str_('12'): 8,
 np.str_('13'): 9,
 np.str_('14'): 10,
 np.str_('15'): 11,
 np.str_('16'): 12,
 np.str_('17'): 13,
 np.str_('187'): 14,
 np.str_('19'): 15,
 np.str_('1939'): 16,
 np.str_('197'): 17,
 np.str_('1972'): 18,
 np.str_('1984'): 19,
 np.str_('1985'): 20,
 np.str_('1992'): 21,
 np.str_('1996'): 22,
 np.str_('1998'): 23,
 np.str_('1999'): 24,
 np.str_('2'): 25,
 np.str_('20'): 26,
 np.str_('200'): 27,
 np.str_('2000'): 28,
 np.str_('2001'): 29,
 np.str_('2002'): 30,
 np.str_('2003'): 31,
 np.str_('2004'): 32,
 np.str_('2006'): 33,
 np.str_('2007'): 34,
 np.str_('2008'): 35,
 np.str_('2009'): 36,
 np.str_('2010'): 37,
 np.str_('2011'): 38,
 np.str_('2012'): 39,
 np.str_('2013'): 40,
 np.str_('2014'): 41,
 np.str_('2015'): 42,
 np.str_('2016'): 43,
 np.str_('2020'): 44,
 np.str_('21'): 45,
 np.str_('21st'): 46,
 np.str_('227

In [ ]:
n_words = 10
input_words = []
next_word = []

for i in range(len(tokens) - n_words) :
  input_words.append(tokens[i:i + n_words])
  next_word.append(tokens[i + n_words])

In [ ]:
# for each sample, n input words and then a boolean for each possible next word
X = np.zeros((len(input_words) , n_words , len(unique_tokens)) , dtype = bool)

In [ ]:
 # for each sample a boolean for each possible next word
y = np.zeros((len(next_word), len(unique_tokens)) , dtype = bool)

In [ ]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_word[i]]] = 1

In [ ]:
X

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [ ]:
y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
optimizer = RMSprop(learning_rate = 0.01)
model.compile(loss = "categorical_crossentropy" , optimizer = optimizer , metrics = ["accuracy"])
history = model.fit(X , y , batch_size = 128 , epochs=25 , shuffle = True).history

Epoch 1/25
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0448 - loss: 7.0240
Epoch 2/25
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0601 - loss: 6.5730
Epoch 3/25
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0650 - loss: 6.4624
Epoch 4/25
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0729 - loss: 6.2906
Epoch 5/25
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0777 - loss: 6.1342
Epoch 6/25
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0826 - loss: 6.0404
Epoch 7/25
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0930 - loss: 5.8337
Epoch 8/25
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1102 - loss: 5.5263
Epoch 9/25
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.1247 - loss: 5.3133
Epoch 10/25
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1537 - loss: 5.0026
Epoch 11/25
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.1838 - loss: 4.6890
Epoch 12/25
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy:

In [ ]:
model.save("text_gen_model2.h5")
with open("history2.p" , "wb") as f :
  pickle.dump(history , f)

In [ ]:
model = load_model("text_gen_model2.h5")
history = pickle.load(open("history2.p", "rb"))

In [ ]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower().split()

    # Ensure fixed input length
    input_text = input_text[-n_words:]  # Trim to last n_words
    X = np.zeros((1, n_words, len(unique_tokens)))

    for i, word in enumerate(input_text):
        index = unique_token_index.get(word)
        if index is not None:
            X[0, i, index] = 1

    predictions = model.predict(X, verbose=0)[0]  # Shape: (vocab_size,)

    # Get top `n_best` indices with highest probabilities
    top_indices = np.argsort(predictions)[-n_best:][::-1]

    return top_indices  # List of token indices

In [ ]:
possible = predict_next_word("I will have to look into this things because I" , 5)

In [ ]:
top_indices = predict_next_word("some input", 5)
for idx in top_indices:
    print(unique_tokens[idx])

trump
he
i
but
after


In [ ]:
def generate_text(input_text, n_words, creativity=3, context_window=5):
    word_sequence = tokenizer.tokenize(input_text.lower())  # Tokenize once

    for _ in range(n_words):
        # Get the last `context_window` tokens as input context
        sub_sequence = " ".join(word_sequence[-context_window:])

        try:
            top_indices = predict_next_word(sub_sequence, creativity)  # Get top-k indices
            predicted_index = random.choice(top_indices)               # Pick one randomly
            choice = unique_tokens[predicted_index]                    # Convert to word
        except:
            choice = random.choice(unique_tokens)                      # Fallback: pick random word

        word_sequence.append(choice)

    return " ".join(word_sequence)

In [ ]:
generate_text("I will have to look into things because  I",50,10,)

'i will have to look into things because i and them obama record and bush bush allows inaccurate democrats of now while is but home some all high while and supporting with facetime at is his her who a or with his at office out i he her was he happened bush happened with when when more democrats the'

In [ ]:
  generate_text("The president of the United States announced yesterday that he", 100, 10)

'the president of the united states announced yesterday that he 2016 but problems reason they has now personal stop among at you a i her her had old at them over if would it also have there his listen your sen times is this her that a veteran uranium supporting if and here with with women after paris years grade email up being a obama all all energy leadership republican there enough days centrifuges on along 000 all that those their being them them doctor evidence there donald trump happened like crisscrossing up obama 2016 later uranium home men him any which supporting is not it has but if trump'

In [ ]:
for idx in predict_next_word("The president will most likely not be there to help", 5):
    print(unique_tokens[idx])

years
of
all
how
version


In [ ]:
  generate_text("The man behind me", 15, 2)

'the man behind me that hillary been more no a a a could changed nationalism office bush supporting trump'